#### This notebook uses *datenguidepy* to collect regional statistics from the German Federal Statistical Office.

You can compare the result to [Regionalatlas Statistisches Bundesamt](https://www-genesis.destatis.de/gis/genView?GenMLURL=https://www-genesis.destatis.de/regatlas/AI002-1.xml&CONTEXT=REGATLAS01).

Steps:

- Load package *datenguidepy*

- Get region code on nuts1 level with "get_all_regions().query('level == nuts1')"

- Search for statistics codes with "get_statistics().query('XXX')"

- Define a year filter with "add_args({'year' : [2015]})"

- Define Query using statistics codes, region code and add a filter

- Collect data

- Add data on the shape of the regions first

- Save as geojson

In [1]:
from datenguidepy import Query

from datenguidepy.query_helper import get_all_regions, get_statistics

In [2]:
reg = get_all_regions()

relevant_region_subset = reg.query('level == "nuts1"').index

In [3]:
def get_data(regcode, stat):
    query = Query.region(regcode)
    field = query.add_field(stat)
    field.add_args({'year': [2015]})
    df = query.results()
    return df

In [4]:
print(get_statistics().query('short_description.str.contains("Geburten-/Gestorbenenüberschuss je 10.000")', engine='python'))

print(' ------------- ')

print(get_statistics().query('short_description.str.contains("Lebendgeborene je 10.000 Einwohner")', engine='python'))

   statistics                                short_description  \
56     AI0211  Geburten-/Gestorbenenüberschuss je 10.000 Einw.   

                                     long_description  
56  **Geburten-/Gestorbenenüberschuss je 10.000 Ei...  
 ------------- 
   statistics                   short_description  \
54     AI0209  Lebendgeborene je 10.000 Einwohner   

                                     long_description  
54  **Lebendgeborene je 10.000 Einwohner**\n*aus G...  


In [5]:
import pandas as pd

dataout = pd.DataFrame()

statlist = ['AI0211', 'AI0209']

for stat in statlist:
    data = pd.DataFrame()
    for regcode in relevant_region_subset:
        df = get_data(str(regcode), stat)
        if df.empty:
            print("no data for region code " + regcode + " and statistic " + stat)
            pass
        else:
            data = data.append(df).copy()
    if dataout.empty:
        dataout = data.copy()
    else:
        dataout = dataout.merge(data, on = ['id', 'name', 'year']).copy()
        
dataout.head()

,id,name,AI0211,year,AI0211_value,AI0211_source_title_de,AI0211_source_valid_from,AI0211_source_periodicity,AI0211_source_name,AI0211_source_url,AI0209,AI0209_value,AI0209_source_title_de,AI0209_source_valid_from,AI0209_source_periodicity,AI0209_source_name,AI0209_source_url
0,10,Saarland,"{'year': 2015, 'value': -59.6, 'source': {'tit...",2015,-59.6,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None,"{'year': 2015, 'value': 75.7, 'source': {'titl...",75.7,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None
1,11,Berlin,"{'year': 2015, 'value': 10.7, 'source': {'titl...",2015,10.7,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None,"{'year': 2015, 'value': 108.8, 'source': {'tit...",108.8,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None
2,12,Brandenburg,"{'year': 2015, 'value': -47.1, 'source': {'tit...",2015,-47.1,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None,"{'year': 2015, 'value': 77.3, 'source': {'titl...",77.3,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None
3,13,Mecklenburg-Vorpommern,"{'year': 2015, 'value': -43.7, 'source': {'tit...",2015,-43.7,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None,"{'year': 2015, 'value': 82.8, 'source': {'titl...",82.8,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None
4,14,Sachsen,"{'year': 2015, 'value': -44.2, 'source': {'tit...",2015,-44.2,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None,"{'year': 2015, 'value': 89.6, 'source': {'titl...",89.6,Regionalatlas Deutschland,1995-01-01T00:00:00,JAEHRLICH,99910,None


Add information on the shape of the regions ([downloaded from](https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete/nuts-gebiete-1-250-000-stand-01-01-nuts250-01-01.html)):



In [ ]:
path = ADD YOUR PATH

In [17]:
import geopandas as gpd

de = gpd.read_file(path + '/nuts250_2019-01-01.gk3-2.shape/nuts250/250_NUTS1.shp', encoding = 'UTF-8')

de.rename(columns = {'NUTS_NAME': 'name'}, inplace = True)

de = de.to_crs({'init': 'epsg:4326'})

df = (gpd.GeoDataFrame(de.merge(dataout, on = 'name'))
      .drop_duplicates(subset=['id', 'year'])
      [['geometry',  'name', 'year',  'AI0211_value', 'AI0209_value']])

In [18]:
from shapely.geometry.polygon import Polygon

from shapely.geometry.multipolygon import MultiPolygon

df["geometry"] = [MultiPolygon([feature]) if type(feature) == Polygon \
    else feature for feature in df["geometry"]]

df.to_file(path + "nuts1.geojson", driver='GeoJSON')